In [ ]:
!pip install matplotlib_inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')
import scipy.stats as ss
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
import pandas as pd

**Задание 1**

In [ ]:
data = pd.read_csv("body_temp.csv")
temp_sample = np.array(data["body temperature"])
n = 130
alpha = 0.05
t_ = ss.t(n-1).ppf(1 - alpha/2)
print(np.sqrt(n) * (temp_sample.mean() - 36.6) / temp_sample.std(ddof=1))
(temp_sample.mean() - t_ * temp_sample.std(ddof=1) / np.sqrt(n), temp_sample.mean() + t_ * temp_sample.std(ddof=1) / np.sqrt(n))

5.812396136123533


(36.7369943876206, 36.878390227764015)

**Задание 2**

In [ ]:
def g(p):
  return -2*np.arcsin(np.sqrt(1 - p))

n = 100
z = np.sqrt(100) * abs(g(0.04) - g(0.1))
2 * (1 - ss.norm().cdf(z))

0.01604665515437853

**Задание 3**

In [ ]:
sample = np.array(list(map(float, input().split())))
n = len(sample)

def T(sample):
  return  ss.cauchy.logpdf(sample).mean() - ss.norm.logpdf(sample).mean()

t = T(sample)

N = 10000
samples = ss.norm.rvs(size=(N,n))
print(np.array([T(sample) > t for sample in samples]).mean())

**Задание 4**

$$
\tau(x_{[1]}) = \begin{cases}
   1 &\text{if }  p(x_{[1]} | H_1) > cp(x_{[1]}|H_0)\\
   0 &\text{else }  
\end{cases}
$$
где $\alpha(\tau(x_{[1]})) = \alpha = x$. Тогда лемме Неймана—Пирсона для всех критериев $\tau'$ таких, что
$\alpha(\tau'(x_{[1]})) \leq \alpha$ верно:

$\beta(\tau(x_{[1]})) \geq \beta(\tau'(x_{[1]}))$.

Т.е. $\tau$ наиболее мощный критерий.

Известно, что

$$
\mathbb{P}(\tau(x_{[1]}) = 1 | H_0) = \alpha \Rightarrow (\tau(x_{[1]}) = 1 \Leftrightarrow x_{[1]} \in [0,\alpha) \cup (1,+\infty) )
$$
Тогда
$$
\mathbb{P}(\tau(x_{[1]}) = 0 | H_1) = \mathbb{P}(x_{[1]} \in (\alpha, 1] | H_1)
= \int_{\alpha}^{1}e^{-x}dx = e^{-\alpha} - e^{-1}
$$
Тогда 
$$
\beta(\tau(x_{[1]}) = 1 - (e^{-\alpha} - e^{-1})
$$

**Задание 5**

In [ ]:
np.random.seed(7)
alpha = 0.05
beta = 0.05
a = np.log(beta / (1 - alpha))
b = np.log((1 - beta) / alpha)
N_ = 100000
step = 100

##########

nums = np.array([1] * N_)
logL = 0
S = np.zeros(N_)
cnt = 0
N = N_
j = 0
while (nums == j * step + 1).any():
    N = sum(nums == j * step + 1)
    ind = nums == j * step + 1
    samples = ss.norm().rvs(size=(N, step))
    logL = np.array([S[ind]] * step).T + np.cumsum(ss.norm(0.5).logpdf(samples) - ss.norm.logpdf(samples), axis=1)
    S[ind] = logL[:, -1]
    nums[ind] += (np.cumsum(1 * (a > logL) + 2 * (logL > b), axis=1) == 0).sum(axis=1)
    j += 1
    for k, num in enumerate((np.cumsum(1 * (a > logL) + 2 * (logL > b), axis=1) == 0).sum(axis=1)):
        if num != step:
            cnt += (logL[k, num] > a)

print(cnt/N_)

###########

nums = np.array([1] * N_)
logL = 0
S = np.zeros(N_)
cnt = 0
N = N_
j = 0
while (nums == j * step + 1).any():
    N = sum(nums == j * step + 1)
    ind = nums == j * step + 1
    samples = ss.norm(0.5).rvs(size=(N, step))
    logL = np.array([S[ind]] * step).T + np.cumsum(ss.norm(0.5).logpdf(samples) - ss.norm.logpdf(samples), axis=1)
    S[ind] = logL[:, -1]
    nums[ind] += (np.cumsum(1 * (a > logL) + 2 * (logL > b), axis=1) == 0).sum(axis=1)
    j += 1
    for k, num in enumerate((np.cumsum(1 * (a > logL) + 2 * (logL > b), axis=1) == 0).sum(axis=1)):
        if num != step:
            cnt += (logL[k, num] < b)

print(cnt/N_)

0.03865
0.0375
